In [19]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Set your image dimensions
img_height, img_width = 90, 90
img_channels = 3  # For RGB images

# Set your dataset paths within Colab
base_path = '/content/'  # Adjust this path

# Define paths for training, validation, and testing
train_folder = 'Train'
valid_folder = 'Valid'
test_folder = 'Test'

train_path = os.path.join(base_path, train_folder)
valid_path = os.path.join(base_path, valid_folder)
test_path = os.path.join(base_path, test_folder)

# Print paths for debugging
print("Train path:", train_path)
print("Validation path:", valid_path)
print("Test path:", test_path)


test_datagen = ImageDataGenerator(rescale=1./255)

# Specify class names
class_names = ['Abnormal', 'Normal']

# Train data generator
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names
)

# Validation data generator
validation_generator = test_datagen.flow_from_directory(
    valid_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names
)

# Define the CNN model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_generator,
    epochs=30,  # Increase or decrease as needed
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Assuming you have a 'Test' folder for evaluation
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}')


Train path: /content/Train
Validation path: /content/Valid
Test path: /content/Test
Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Epoch 1/30
38/38 [==============================] - 18s 433ms/step - loss: 0.7013 - accuracy: 0.5550 - val_loss: 0.7025 - val_accuracy: 0.5017
Epoch 2/30
38/38 [==============================] - 15s 409ms/step - loss: 0.6779 - accuracy: 0.5783 - val_loss: 0.6416 - val_accuracy: 0.6567
Epoch 3/30
38/38 [==============================] - 15s 392ms/step - loss: 0.6706 - accuracy: 0.6067 - val_loss: 0.6268 - val_accuracy: 0.6867
Epoch 4/30
38/38 [==============================] - 15s 389ms/step - loss: 0.6295 - accuracy: 0.6633 - val_loss: 0.5643 - val_accuracy: 0.6833
Epoch 5/30
38/38 [==============================] - 14s 374ms/step - loss: 0.5976 - accuracy: 0.6817 - val_loss: 0.5543 - val_accuracy: 0.7483
Epoch 6/30
38/38 [==============================] - 15s 380ms/step - loss: 0.5736 - accuracy: 0.7217 - val_loss: 0.5372

In [21]:
# ... (your training code)

# Save the trained model
model.save('/content/trained_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
